# Exploración Exhaustiva de Datos - Pruebas Saber Pro Colombia

## Descripción del Dataset

Este notebook presenta una exploración detallada y exhaustiva de los datos de las Pruebas Saber Pro en Colombia. El dataset contiene información sobre estudiantes universitarios y su rendimiento académico, incluyendo:

- **Variable Objetivo**: `RENDIMIENTO_GLOBAL` (bajo, medio-bajo, medio-alto, alto)
- **Variables Demográficas**: Departamento, periodo académico
- **Variables Académicas**: Programa académico, indicadores de rendimiento
- **Variables Socioeconómicas**: Estrato, educación de padres, recursos del hogar
- **Variables Laborales**: Horas semanales de trabajo

**Objetivo**: Explorar en profundidad el dataset para entender patrones, relaciones y características de los datos.

---
## 1. Configuración Inicial y Carga de Datos

In [ ]:
# Descargar archivos de inicialización
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init
init.init(force_download=False)
init.get_weblink()

### 1.1 Descargar datos desde Kaggle

**Instrucciones:**
1. Crear archivo `kaggle.json` con tu token de autenticación
2. En Kaggle: click en icono de usuario (arriba-derecha) → Settings → API → Create New Token
3. Subir el archivo a este workspace
4. Ejecutar la siguiente celda

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

In [ ]:
# Descomprimir archivos
!unzip -o udea*.zip > /dev/null
!ls -lh *.csv

### 1.2 Importar librerías necesarias

In [ ]:
# Librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rlxutils import subplots

# Configuración de visualizaciones
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Opciones de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Configuración de warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")

### 1.3 Cargar datasets

In [ ]:
# Cargar datos de entrenamiento
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(f"Dimensiones del conjunto de entrenamiento: {train.shape}")
print(f"Dimensiones del conjunto de prueba: {test.shape}")
print(f"\nTotal de registros: {train.shape[0] + test.shape[0]:,}")

In [ ]:
# Primeras filas del dataset de entrenamiento
print("=" * 80)
print("PRIMERAS 5 FILAS DEL DATASET DE ENTRENAMIENTO")
print("=" * 80)
train.head()

In [ ]:
# Información general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
train.info()

---
## 2. Análisis de Calidad de Datos y Valores Faltantes

### 2.1 Resumen de valores faltantes

In [ ]:
# Calcular valores faltantes para cada columna
missing_data = pd.DataFrame({
    'Columna': train.columns,
    'Valores_Faltantes': train.isnull().sum(),
    'Porcentaje': (train.isnull().sum() / len(train) * 100).round(2)
}).sort_values('Valores_Faltantes', ascending=False)

missing_data = missing_data[missing_data['Valores_Faltantes'] > 0]

print("=" * 80)
print("ANÁLISIS DE VALORES FALTANTES")
print("=" * 80)
if len(missing_data) > 0:
    print(missing_data.to_string(index=False))
    print(f"\nTotal de columnas con valores faltantes: {len(missing_data)}")
else:
    print("No hay valores faltantes en el dataset")

In [ ]:
# Visualización de valores faltantes
if len(missing_data) > 0:
    plt.figure(figsize=(12, 6))
    plt.barh(missing_data['Columna'], missing_data['Porcentaje'], color='coral')
    plt.xlabel('Porcentaje de Valores Faltantes (%)', fontsize=12)
    plt.ylabel('Columna', fontsize=12)
    plt.title('Porcentaje de Valores Faltantes por Columna', fontsize=14, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No hay valores faltantes para visualizar")

### 2.2 Verificación de duplicados

In [ ]:
# Verificar duplicados
duplicados = train.duplicated().sum()
duplicados_id = train['ID'].duplicated().sum()

print("=" * 80)
print("ANÁLISIS DE DUPLICADOS")
print("=" * 80)
print(f"Filas completamente duplicadas: {duplicados}")
print(f"IDs duplicados: {duplicados_id}")
print(f"IDs únicos: {train['ID'].nunique():,}")

### 2.3 Tipos de datos por columna

In [ ]:
# Resumen de tipos de datos
tipos_datos = pd.DataFrame({
    'Columna': train.columns,
    'Tipo_Dato': train.dtypes.values,
    'Valores_Únicos': [train[col].nunique() for col in train.columns]
})

print("=" * 80)
print("TIPOS DE DATOS Y CARDINALIDAD")
print("=" * 80)
print(tipos_datos.to_string(index=False))

---
## 3. Análisis de la Variable Objetivo: RENDIMIENTO_GLOBAL

In [ ]:
# Distribución de la variable objetivo
print("=" * 80)
print("DISTRIBUCIÓN DE RENDIMIENTO_GLOBAL")
print("=" * 80)

rendimiento_counts = train['RENDIMIENTO_GLOBAL'].value_counts()
rendimiento_pct = train['RENDIMIENTO_GLOBAL'].value_counts(normalize=True) * 100

resumen_rendimiento = pd.DataFrame({
    'Categoría': rendimiento_counts.index,
    'Cantidad': rendimiento_counts.values,
    'Porcentaje': rendimiento_pct.values.round(2)
})

print(resumen_rendimiento.to_string(index=False))

In [ ]:
# Visualización de la variable objetivo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ordenar las categorías lógicamente
orden_rendimiento = ['bajo', 'medio-bajo', 'medio-alto', 'alto']
colores = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']

# Gráfico de barras
counts_ordenado = train['RENDIMIENTO_GLOBAL'].value_counts()[orden_rendimiento]
axes[0].bar(orden_rendimiento, counts_ordenado.values, color=colores, edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('Categoría de Rendimiento', fontsize=12)
axes[0].set_ylabel('Cantidad de Estudiantes', fontsize=12)
axes[0].set_title('Distribución de Rendimiento Global', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Agregar etiquetas con valores
for i, v in enumerate(counts_ordenado.values):
    axes[0].text(i, v + 2000, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Gráfico de pastel
axes[1].pie(counts_ordenado.values, labels=orden_rendimiento, autopct='%1.1f%%', 
            colors=colores, startangle=90, explode=[0.05, 0.05, 0.05, 0.05])
axes[1].set_title('Proporción de Rendimiento Global', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 4. Análisis Univariado: Variables Categóricas

### 4.1 Periodo Académico

In [ ]:
print("=" * 80)
print("PERIODO_ACADEMICO")
print("=" * 80)

periodo_stats = pd.DataFrame({
    'Periodo': train['PERIODO_ACADEMICO'].value_counts().index,
    'Cantidad': train['PERIODO_ACADEMICO'].value_counts().values,
    'Porcentaje': (train['PERIODO_ACADEMICO'].value_counts(normalize=True) * 100).round(2).values
}).sort_values('Periodo')

print(periodo_stats.to_string(index=False))

# Visualización
plt.figure(figsize=(10, 5))
plt.bar(periodo_stats['Periodo'].astype(str), periodo_stats['Cantidad'], color='steelblue', edgecolor='black')
plt.xlabel('Periodo Académico', fontsize=12)
plt.ylabel('Número de Estudiantes', fontsize=12)
plt.title('Distribución de Estudiantes por Periodo Académico', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.2 Estrato Socioeconómico

In [ ]:
print("=" * 80)
print("F_ESTRATOVIVIENDA")
print("=" * 80)

estrato_stats = pd.DataFrame({
    'Estrato': train['F_ESTRATOVIVIENDA'].value_counts().index,
    'Cantidad': train['F_ESTRATOVIVIENDA'].value_counts().values,
    'Porcentaje': (train['F_ESTRATOVIVIENDA'].value_counts(normalize=True) * 100).round(2).values
})

print(estrato_stats.to_string(index=False))
print(f"\nValores faltantes: {train['F_ESTRATOVIVIENDA'].isnull().sum()}")

# Visualización
plt.figure(figsize=(10, 5))

# Función para extraer el número del estrato o asignar 0 para "Sin Estrato"
def extract_estrato_num(estrato_str):
    parts = str(estrato_str).split()
    if len(parts) >= 2 and parts[-1].isdigit():
        return int(parts[-1])
    return 0  # Para "Sin Estrato" u otros valores

estrato_order = sorted([e for e in train['F_ESTRATOVIVIENDA'].dropna().unique()],
                       key=extract_estrato_num)
estrato_counts = train['F_ESTRATOVIVIENDA'].value_counts()[estrato_order]
plt.bar(range(len(estrato_counts)), estrato_counts.values, color='teal', edgecolor='black')
plt.xticks(range(len(estrato_counts)), estrato_order, rotation=0)
plt.xlabel('Estrato', fontsize=12)
plt.ylabel('Número de Estudiantes', fontsize=12)
plt.title('Distribución de Estudiantes por Estrato Socioeconómico', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.3 Educación de la Madre

In [ ]:
print("=" * 80)
print("F_EDUCACIONMADRE")
print("=" * 80)

edu_madre_stats = pd.DataFrame({
    'Nivel_Educativo': train['F_EDUCACIONMADRE'].value_counts().index,
    'Cantidad': train['F_EDUCACIONMADRE'].value_counts().values,
    'Porcentaje': (train['F_EDUCACIONMADRE'].value_counts(normalize=True) * 100).round(2).values
})

print(edu_madre_stats.to_string(index=False))
print(f"\nValores faltantes: {train['F_EDUCACIONMADRE'].isnull().sum()}")

# Visualización
plt.figure(figsize=(12, 6))
plt.barh(range(len(edu_madre_stats)), edu_madre_stats['Cantidad'], color='indianred', edgecolor='black')
plt.yticks(range(len(edu_madre_stats)), edu_madre_stats['Nivel_Educativo'])
plt.xlabel('Número de Estudiantes', fontsize=12)
plt.ylabel('Nivel Educativo de la Madre', fontsize=12)
plt.title('Distribución del Nivel Educativo de la Madre', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.4 Educación del Padre

In [ ]:
print("=" * 80)
print("F_EDUCACIONPADRE")
print("=" * 80)

edu_padre_stats = pd.DataFrame({
    'Nivel_Educativo': train['F_EDUCACIONPADRE'].value_counts().index,
    'Cantidad': train['F_EDUCACIONPADRE'].value_counts().values,
    'Porcentaje': (train['F_EDUCACIONPADRE'].value_counts(normalize=True) * 100).round(2).values
})

print(edu_padre_stats.to_string(index=False))
print(f"\nValores faltantes: {train['F_EDUCACIONPADRE'].isnull().sum()}")

# Visualización
plt.figure(figsize=(12, 6))
plt.barh(range(len(edu_padre_stats)), edu_padre_stats['Cantidad'], color='royalblue', edgecolor='black')
plt.yticks(range(len(edu_padre_stats)), edu_padre_stats['Nivel_Educativo'])
plt.xlabel('Número de Estudiantes', fontsize=12)
plt.ylabel('Nivel Educativo del Padre', fontsize=12)
plt.title('Distribución del Nivel Educativo del Padre', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.5 Valor de Matrícula Universitaria

In [ ]:
print("=" * 80)
print("E_VALORMATRICULAUNIVERSIDAD")
print("=" * 80)

matricula_stats = pd.DataFrame({
    'Rango_Matricula': train['E_VALORMATRICULAUNIVERSIDAD'].value_counts().index,
    'Cantidad': train['E_VALORMATRICULAUNIVERSIDAD'].value_counts().values,
    'Porcentaje': (train['E_VALORMATRICULAUNIVERSIDAD'].value_counts(normalize=True) * 100).round(2).values
})

print(matricula_stats.to_string(index=False))
print(f"\nValores faltantes: {train['E_VALORMATRICULAUNIVERSIDAD'].isnull().sum()}")

# Visualización
plt.figure(figsize=(14, 6))
plt.bar(range(len(matricula_stats)), matricula_stats['Cantidad'], color='darkgreen', edgecolor='black')
plt.xticks(range(len(matricula_stats)), matricula_stats['Rango_Matricula'], rotation=45, ha='right')
plt.xlabel('Rango de Valor de Matrícula', fontsize=12)
plt.ylabel('Número de Estudiantes', fontsize=12)
plt.title('Distribución del Valor de Matrícula Universitaria', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.6 Horas Semanales de Trabajo

In [ ]:
print("=" * 80)
print("E_HORASSEMANATRABAJA")
print("=" * 80)

horas_stats = pd.DataFrame({
    'Rango_Horas': train['E_HORASSEMANATRABAJA'].value_counts().index,
    'Cantidad': train['E_HORASSEMANATRABAJA'].value_counts().values,
    'Porcentaje': (train['E_HORASSEMANATRABAJA'].value_counts(normalize=True) * 100).round(2).values
})

print(horas_stats.to_string(index=False))
print(f"\nValores faltantes: {train['E_HORASSEMANATRABAJA'].isnull().sum()}")

# Visualización
plt.figure(figsize=(10, 5))
plt.bar(range(len(horas_stats)), horas_stats['Cantidad'], color='orange', edgecolor='black')
plt.xticks(range(len(horas_stats)), horas_stats['Rango_Horas'], rotation=45, ha='right')
plt.xlabel('Horas Semanales de Trabajo', fontsize=12)
plt.ylabel('Número de Estudiantes', fontsize=12)
plt.title('Distribución de Horas Semanales de Trabajo', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.7 Recursos del Hogar (Internet, Computador, Lavadora, Automóvil)

In [ ]:
# Variables de recursos del hogar
recursos = ['F_TIENEINTERNET', 'F_TIENECOMPUTADOR', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL']

print("=" * 80)
print("RECURSOS DEL HOGAR")
print("=" * 80)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, recurso in enumerate(recursos):
    recurso_counts = train[recurso].value_counts()
    
    print(f"\n{recurso}:")
    print(f"  Sí: {recurso_counts.get('Si', 0):,} ({recurso_counts.get('Si', 0)/len(train)*100:.2f}%)")
    print(f"  No: {recurso_counts.get('No', 0):,} ({recurso_counts.get('No', 0)/len(train)*100:.2f}%)")
    print(f"  Valores faltantes: {train[recurso].isnull().sum()}")
    
    # Gráfico
    axes[idx].pie([recurso_counts.get('Si', 0), recurso_counts.get('No', 0)], 
                   labels=['Sí', 'No'], autopct='%1.1f%%', 
                   colors=['lightgreen', 'lightcoral'], startangle=90)
    axes[idx].set_title(recurso.replace('F_TIENE', 'Tiene ').replace('_', ' '), 
                        fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

### 4.8 Otras Variables Binarias

In [ ]:
# Variables binarias adicionales
vars_binarias = ['E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO']

print("=" * 80)
print("OTRAS VARIABLES BINARIAS")
print("=" * 80)

for var in vars_binarias:
    var_counts = train[var].value_counts()
    print(f"\n{var}:")
    for categoria, count in var_counts.items():
        print(f"  {categoria}: {count:,} ({count/len(train)*100:.2f}%)")
    print(f"  Valores faltantes: {train[var].isnull().sum()}")

---
## 5. Análisis Univariado: Variables Numéricas (Indicadores)

### 5.1 Estadísticas Descriptivas de los Indicadores

In [ ]:
# Estadísticas descriptivas de los 4 indicadores
indicadores = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS DE LOS INDICADORES")
print("=" * 80)

stats_indicadores = train[indicadores].describe().T
stats_indicadores['missing'] = train[indicadores].isnull().sum()
print(stats_indicadores.to_string())

In [ ]:
# Distribuciones de los indicadores
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, indicador in enumerate(indicadores):
    axes[idx].hist(train[indicador].dropna(), bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].axvline(train[indicador].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {train[indicador].mean():.3f}')
    axes[idx].axvline(train[indicador].median(), color='green', linestyle='--', linewidth=2, label=f'Mediana: {train[indicador].median():.3f}')
    axes[idx].set_xlabel('Valor', fontsize=11)
    axes[idx].set_ylabel('Frecuencia', fontsize=11)
    axes[idx].set_title(f'Distribución de {indicador}', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots de los indicadores
plt.figure(figsize=(12, 6))
train[indicadores].boxplot(figsize=(12, 6))
plt.ylabel('Valor', fontsize=12)
plt.xlabel('Indicador', fontsize=12)
plt.title('Boxplots de los 4 Indicadores', fontsize=14, fontweight='bold')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2 Correlación entre Indicadores

In [ ]:
# Matriz de correlación entre indicadores
corr_indicadores = train[indicadores].corr()

print("=" * 80)
print("MATRIZ DE CORRELACIÓN ENTRE INDICADORES")
print("=" * 80)
print(corr_indicadores.to_string())

# Visualización del heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_indicadores, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación entre Indicadores', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 6. Análisis Geográfico: Distribución por Departamento

In [ ]:
print("=" * 80)
print("E_PRGM_DEPARTAMENTO")
print("=" * 80)

depto_stats = pd.DataFrame({
    'Departamento': train['E_PRGM_DEPARTAMENTO'].value_counts().index,
    'Cantidad': train['E_PRGM_DEPARTAMENTO'].value_counts().values,
    'Porcentaje': (train['E_PRGM_DEPARTAMENTO'].value_counts(normalize=True) * 100).round(2).values
}).head(20)  # Top 20 departamentos

print("Top 20 Departamentos con más estudiantes:")
print(depto_stats.to_string(index=False))
print(f"\nTotal de departamentos únicos: {train['E_PRGM_DEPARTAMENTO'].nunique()}")
print(f"Valores faltantes: {train['E_PRGM_DEPARTAMENTO'].isnull().sum()}")

In [ ]:
# Visualización de top 15 departamentos
top_deptos = train['E_PRGM_DEPARTAMENTO'].value_counts().head(15)

plt.figure(figsize=(12, 6))
plt.barh(range(len(top_deptos)), top_deptos.values, color='mediumpurple', edgecolor='black')
plt.yticks(range(len(top_deptos)), top_deptos.index)
plt.xlabel('Número de Estudiantes', fontsize=12)
plt.ylabel('Departamento', fontsize=12)
plt.title('Top 15 Departamentos con Más Estudiantes', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

---
## 7. Análisis de Programas Académicos

In [ ]:
print("=" * 80)
print("E_PRGM_ACADEMICO")
print("=" * 80)

programa_stats = pd.DataFrame({
    'Programa': train['E_PRGM_ACADEMICO'].value_counts().index,
    'Cantidad': train['E_PRGM_ACADEMICO'].value_counts().values,
    'Porcentaje': (train['E_PRGM_ACADEMICO'].value_counts(normalize=True) * 100).round(2).values
}).head(20)  # Top 20 programas

print("Top 20 Programas Académicos con más estudiantes:")
print(programa_stats.to_string(index=False))
print(f"\nTotal de programas únicos: {train['E_PRGM_ACADEMICO'].nunique()}")
print(f"Valores faltantes: {train['E_PRGM_ACADEMICO'].isnull().sum()}")

In [ ]:
# Visualización de top 15 programas
top_programas = train['E_PRGM_ACADEMICO'].value_counts().head(15)

plt.figure(figsize=(12, 6))
plt.barh(range(len(top_programas)), top_programas.values, color='coral', edgecolor='black')
plt.yticks(range(len(top_programas)), top_programas.index)
plt.xlabel('Número de Estudiantes', fontsize=12)
plt.ylabel('Programa Académico', fontsize=12)
plt.title('Top 15 Programas Académicos con Más Estudiantes', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

---
## 8. Análisis Bivariado: Variables vs RENDIMIENTO_GLOBAL

### 8.1 Estrato Socioeconómico vs Rendimiento

In [ ]:
# Visualización con gráficos de barras apiladas
# Función para extraer el número del estrato o asignar 0 para "Sin Estrato"
def extract_estrato_num(estrato_str):
    parts = str(estrato_str).split()
    if len(parts) >= 2 and parts[-1].isdigit():
        return int(parts[-1])
    return 0  # Para "Sin Estrato" u otros valores

estratos_orden = sorted([e for e in train['F_ESTRATOVIVIENDA'].dropna().unique()],
                        key=extract_estrato_num)

fig, axes = plt.subplots(1, len(estratos_orden), figsize=(16, 5), sharey=True)

for idx, estrato in enumerate(estratos_orden):
    data_estrato = train[train['F_ESTRATOVIVIENDA'] == estrato]
    rendimiento_counts = data_estrato['RENDIMIENTO_GLOBAL'].value_counts()[orden_rendimiento]
    
    axes[idx].bar(orden_rendimiento, rendimiento_counts.values, color=colores, edgecolor='black')
    axes[idx].set_title(estrato, fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Rendimiento', fontsize=10)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(axis='y', alpha=0.3)

axes[0].set_ylabel('Número de Estudiantes', fontsize=11)
fig.suptitle('Distribución de Rendimiento por Estrato Socioeconómico', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 8.2 Educación de la Madre vs Rendimiento

In [ ]:
# Tabla cruzada de Educación Madre vs Rendimiento
print("=" * 80)
print("EDUCACIÓN DE LA MADRE VS RENDIMIENTO GLOBAL")
print("=" * 80)

crosstab_edumadre = pd.crosstab(train['F_EDUCACIONMADRE'], train['RENDIMIENTO_GLOBAL'], normalize='index') * 100
crosstab_edumadre = crosstab_edumadre[orden_rendimiento]
print("\nPorcentaje por fila (cada nivel educativo suma 100%):")
print(crosstab_edumadre.round(2).to_string())

In [ ]:
# Visualización con subplots
niveles_edu_madre = sorted(train['F_EDUCACIONMADRE'].dropna().unique())

for ax, nivel in subplots(niveles_edu_madre, n_cols=4, usizex=4):
    data_nivel = train[train['F_EDUCACIONMADRE'] == nivel]
    rendimiento_counts = data_nivel['RENDIMIENTO_GLOBAL'].value_counts()
    if len(rendimiento_counts) > 0:
        rendimiento_counts[orden_rendimiento].plot(kind='bar', ax=ax, color=colores, edgecolor='black')
    plt.title(nivel, fontsize=10)
    plt.xlabel('')
    plt.ylabel('Cantidad')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 8.3 Educación del Padre vs Rendimiento

In [ ]:
# Tabla cruzada de Educación Padre vs Rendimiento
print("=" * 80)
print("EDUCACIÓN DEL PADRE VS RENDIMIENTO GLOBAL")
print("=" * 80)

crosstab_edupadre = pd.crosstab(train['F_EDUCACIONPADRE'], train['RENDIMIENTO_GLOBAL'], normalize='index') * 100
crosstab_edupadre = crosstab_edupadre[orden_rendimiento]
print("\nPorcentaje por fila (cada nivel educativo suma 100%):")
print(crosstab_edupadre.round(2).to_string())

In [ ]:
# Visualización con subplots
niveles_edu_padre = sorted(train['F_EDUCACIONPADRE'].dropna().unique())

for ax, nivel in subplots(niveles_edu_padre, n_cols=4, usizex=4):
    data_nivel = train[train['F_EDUCACIONPADRE'] == nivel]
    rendimiento_counts = data_nivel['RENDIMIENTO_GLOBAL'].value_counts()
    if len(rendimiento_counts) > 0:
        rendimiento_counts[orden_rendimiento].plot(kind='bar', ax=ax, color=colores, edgecolor='black')
    plt.title(nivel, fontsize=10)
    plt.xlabel('')
    plt.ylabel('Cantidad')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 8.4 Valor de Matrícula vs Rendimiento

In [ ]:
# Tabla cruzada de Valor Matrícula vs Rendimiento
print("=" * 80)
print("VALOR DE MATRÍCULA VS RENDIMIENTO GLOBAL")
print("=" * 80)

crosstab_matricula = pd.crosstab(train['E_VALORMATRICULAUNIVERSIDAD'], train['RENDIMIENTO_GLOBAL'], normalize='index') * 100
crosstab_matricula = crosstab_matricula[orden_rendimiento]
print("\nPorcentaje por fila (cada rango de matrícula suma 100%):")
print(crosstab_matricula.round(2).to_string())

In [ ]:
# Visualización con subplots
rangos_matricula = sorted(train['E_VALORMATRICULAUNIVERSIDAD'].dropna().unique())

for ax, rango in subplots(rangos_matricula, n_cols=4, usizex=4):
    data_rango = train[train['E_VALORMATRICULAUNIVERSIDAD'] == rango]
    rendimiento_counts = data_rango['RENDIMIENTO_GLOBAL'].value_counts()
    if len(rendimiento_counts) > 0:
        rendimiento_counts[orden_rendimiento].plot(kind='bar', ax=ax, color=colores, edgecolor='black')
    plt.title(rango, fontsize=9)
    plt.xlabel('')
    plt.ylabel('Cantidad')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 8.5 Horas de Trabajo vs Rendimiento

In [ ]:
# Tabla cruzada de Horas Trabajo vs Rendimiento
print("=" * 80)
print("HORAS SEMANALES DE TRABAJO VS RENDIMIENTO GLOBAL")
print("=" * 80)

crosstab_horas = pd.crosstab(train['E_HORASSEMANATRABAJA'], train['RENDIMIENTO_GLOBAL'], normalize='index') * 100
crosstab_horas = crosstab_horas[orden_rendimiento]
print("\nPorcentaje por fila (cada rango de horas suma 100%):")
print(crosstab_horas.round(2).to_string())

In [ ]:
# Visualización
rangos_horas = train['E_HORASSEMANATRABAJA'].dropna().unique()

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, rango in enumerate(rangos_horas[:6]):
    data_rango = train[train['E_HORASSEMANATRABAJA'] == rango]
    rendimiento_counts = data_rango['RENDIMIENTO_GLOBAL'].value_counts()
    if len(rendimiento_counts) > 0:
        axes[idx].bar(orden_rendimiento, rendimiento_counts[orden_rendimiento].values, 
                      color=colores, edgecolor='black')
    axes[idx].set_title(str(rango), fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Rendimiento', fontsize=10)
    axes[idx].set_ylabel('Cantidad', fontsize=10)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(axis='y', alpha=0.3)

plt.suptitle('Distribución de Rendimiento por Horas Semanales de Trabajo', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 8.6 Recursos del Hogar vs Rendimiento

In [ ]:
# Análisis de recursos del hogar vs rendimiento
recursos = ['F_TIENEINTERNET', 'F_TIENECOMPUTADOR', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, recurso in enumerate(recursos):
    print("=" * 80)
    print(f"{recurso} VS RENDIMIENTO GLOBAL")
    print("=" * 80)
    
    crosstab_recurso = pd.crosstab(train[recurso], train['RENDIMIENTO_GLOBAL'], normalize='index') * 100
    crosstab_recurso = crosstab_recurso[orden_rendimiento]
    print(crosstab_recurso.round(2).to_string())
    print("\n")
    
    # Visualización
    tiene_si = train[train[recurso] == 'Si']['RENDIMIENTO_GLOBAL'].value_counts()[orden_rendimiento]
    tiene_no = train[train[recurso] == 'No']['RENDIMIENTO_GLOBAL'].value_counts()[orden_rendimiento]
    
    x = np.arange(len(orden_rendimiento))
    width = 0.35
    
    axes[idx].bar(x - width/2, tiene_si.values, width, label='Sí tiene', color='lightgreen', edgecolor='black')
    axes[idx].bar(x + width/2, tiene_no.values, width, label='No tiene', color='lightcoral', edgecolor='black')
    axes[idx].set_xlabel('Rendimiento Global', fontsize=11)
    axes[idx].set_ylabel('Cantidad', fontsize=11)
    axes[idx].set_title(recurso.replace('F_TIENE', 'Tiene '), fontsize=12, fontweight='bold')
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(orden_rendimiento, rotation=45)
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 8.7 Indicadores Numéricos vs Rendimiento

In [ ]:
# Boxplots de indicadores por categoría de rendimiento
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, indicador in enumerate(indicadores):
    data_boxplot = [train[train['RENDIMIENTO_GLOBAL'] == cat][indicador].dropna() 
                    for cat in orden_rendimiento]
    
    bp = axes[idx].boxplot(data_boxplot, labels=orden_rendimiento, patch_artist=True)
    
    # Colorear las cajas
    for patch, color in zip(bp['boxes'], colores):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    
    axes[idx].set_xlabel('Rendimiento Global', fontsize=11)
    axes[idx].set_ylabel('Valor del Indicador', fontsize=11)
    axes[idx].set_title(f'{indicador} por Rendimiento Global', fontsize=12, fontweight='bold')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Estadísticas descriptivas de indicadores por rendimiento
print("=" * 80)
print("ESTADÍSTICAS DE INDICADORES POR RENDIMIENTO GLOBAL")
print("=" * 80)

for indicador in indicadores:
    print(f"\n{indicador}:")
    print("-" * 80)
    stats_por_rendimiento = train.groupby('RENDIMIENTO_GLOBAL')[indicador].describe()[['mean', 'std', 'min', 'max']]
    stats_por_rendimiento = stats_por_rendimiento.reindex(orden_rendimiento)
    print(stats_por_rendimiento.to_string())

---
## 9. Análisis Geográfico Detallado: Rendimiento por Departamento

In [ ]:
# Top 10 departamentos por rendimiento alto
print("=" * 80)
print("ANÁLISIS DE RENDIMIENTO POR DEPARTAMENTO")
print("=" * 80)

# Calcular porcentaje de rendimiento alto por departamento
rendimiento_por_depto = train.groupby('E_PRGM_DEPARTAMENTO')['RENDIMIENTO_GLOBAL'].apply(
    lambda x: (x == 'alto').sum() / len(x) * 100
).sort_values(ascending=False)

# Filtrar departamentos con al menos 100 estudiantes
deptos_grandes = train['E_PRGM_DEPARTAMENTO'].value_counts()
deptos_grandes = deptos_grandes[deptos_grandes >= 100].index
rendimiento_por_depto_filtrado = rendimiento_por_depto[rendimiento_por_depto.index.isin(deptos_grandes)]

print("\nTop 15 Departamentos con Mayor Porcentaje de Rendimiento ALTO (mín. 100 estudiantes):")
print(rendimiento_por_depto_filtrado.head(15).to_string())

print("\n\nTop 15 Departamentos con Menor Porcentaje de Rendimiento ALTO (mín. 100 estudiantes):")
print(rendimiento_por_depto_filtrado.tail(15).to_string())

In [ ]:
# Visualización de top 10 y bottom 10 departamentos
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 10
top_10_deptos = rendimiento_por_depto_filtrado.head(10)
axes[0].barh(range(len(top_10_deptos)), top_10_deptos.values, color='darkgreen', edgecolor='black')
axes[0].set_yticks(range(len(top_10_deptos)))
axes[0].set_yticklabels(top_10_deptos.index)
axes[0].set_xlabel('% Rendimiento Alto', fontsize=12)
axes[0].set_title('Top 10 Departamentos con Mayor % de Rendimiento Alto', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
axes[0].invert_yaxis()

# Bottom 10
bottom_10_deptos = rendimiento_por_depto_filtrado.tail(10)
axes[1].barh(range(len(bottom_10_deptos)), bottom_10_deptos.values, color='darkred', edgecolor='black')
axes[1].set_yticks(range(len(bottom_10_deptos)))
axes[1].set_yticklabels(bottom_10_deptos.index)
axes[1].set_xlabel('% Rendimiento Alto', fontsize=12)
axes[1].set_title('Bottom 10 Departamentos con Menor % de Rendimiento Alto', fontsize=13, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Distribución completa de rendimiento por departamento (top 10 departamentos con más estudiantes)
top_10_deptos_cantidad = train['E_PRGM_DEPARTAMENTO'].value_counts().head(10)

fig, axes = plt.subplots(2, 5, figsize=(18, 8))
axes = axes.flatten()

for idx, depto in enumerate(top_10_deptos_cantidad.index):
    data_depto = train[train['E_PRGM_DEPARTAMENTO'] == depto]
    rendimiento_counts = data_depto['RENDIMIENTO_GLOBAL'].value_counts()[orden_rendimiento]
    
    axes[idx].bar(orden_rendimiento, rendimiento_counts.values, color=colores, edgecolor='black')
    axes[idx].set_title(f"{depto}\n(n={len(data_depto)})", fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Rendimiento', fontsize=9)
    axes[idx].set_ylabel('Cantidad', fontsize=9)
    axes[idx].tick_params(axis='x', rotation=45, labelsize=8)
    axes[idx].grid(axis='y', alpha=0.3)

plt.suptitle('Distribución de Rendimiento en los 10 Departamentos con Más Estudiantes', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 10. Análisis Temporal: Evolución por Periodo Académico

In [ ]:
# Evolución del rendimiento por periodo académico
print("=" * 80)
print("EVOLUCIÓN DEL RENDIMIENTO POR PERIODO ACADÉMICO")
print("=" * 80)

# Tabla cruzada de Periodo vs Rendimiento
crosstab_periodo = pd.crosstab(train['PERIODO_ACADEMICO'], train['RENDIMIENTO_GLOBAL'], normalize='index') * 100
crosstab_periodo = crosstab_periodo[orden_rendimiento]
crosstab_periodo = crosstab_periodo.sort_index()

print("\nPorcentaje de cada categoría de rendimiento por periodo:")
print(crosstab_periodo.round(2).to_string())

In [ ]:
# Visualización de evolución temporal
plt.figure(figsize=(14, 6))

for categoria, color in zip(orden_rendimiento, colores):
    plt.plot(crosstab_periodo.index.astype(str), crosstab_periodo[categoria], 
             marker='o', linewidth=2, label=categoria, color=color)

plt.xlabel('Periodo Académico', fontsize=12)
plt.ylabel('Porcentaje (%)', fontsize=12)
plt.title('Evolución del Rendimiento Global por Periodo Académico', fontsize=14, fontweight='bold')
plt.legend(title='Rendimiento', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Evolución de los indicadores por periodo
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, indicador in enumerate(indicadores):
    indicador_por_periodo = train.groupby('PERIODO_ACADEMICO')[indicador].mean().sort_index()
    
    axes[idx].plot(indicador_por_periodo.index.astype(str), indicador_por_periodo.values, 
                   marker='o', linewidth=2, color='steelblue')
    axes[idx].set_xlabel('Periodo Académico', fontsize=11)
    axes[idx].set_ylabel('Valor Promedio', fontsize=11)
    axes[idx].set_title(f'Evolución de {indicador} por Periodo', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## 11. Análisis de Outliers y Valores Atípicos

In [ ]:
# Detección de outliers en los indicadores usando IQR
print("=" * 80)
print("DETECCIÓN DE OUTLIERS EN INDICADORES (Método IQR)")
print("=" * 80)

outliers_summary = []

for indicador in indicadores:
    Q1 = train[indicador].quantile(0.25)
    Q3 = train[indicador].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train[(train[indicador] < lower_bound) | (train[indicador] > upper_bound)]
    n_outliers = len(outliers)
    pct_outliers = (n_outliers / len(train)) * 100
    
    outliers_summary.append({
        'Indicador': indicador,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'Límite_Inferior': lower_bound,
        'Límite_Superior': upper_bound,
        'N_Outliers': n_outliers,
        'Porcentaje_Outliers': pct_outliers
    })

outliers_df = pd.DataFrame(outliers_summary)
print(outliers_df.to_string(index=False))